In [ ]:
import ISRAnalysis as an
import sys
import ROOT as rt

# unfolding

In [ ]:
doSys = False

channel_postfix = "detector_preFSR_skflat"
year = "2017"

#variable = "[dimass-dipt]"
#bin_def = ["[folded_dimass55-folded_extended]",
#           "[unfolded_dimass55-unfolded_extended]"]

variable = "[dipt-dimass]"
bin_def = ["[folded_extended-folded_dimass55]",
           "[unfolded_extended-unfolded_dimass55]"]

channel = "ee"
bias = 0

# Select regularization mode
regularisation_mode = rt.TUnfold.kRegModeNone

#Select density mode
density_mode = rt.TUnfoldDensity.kDensityModeNone

# Redirect stdout and stderr
sys.stderr = open("./log/stderr_"+channel+"_"+year+".txt", "w")
sys.stdout = open("./log/stdout_"+channel+"_"+year+".txt", "w")

detUnfold = an.ISRAnalysis("DetUNFOLD", # prefix for output root file
                           year, 
                           channel, 
                           channel_postfix,
                           "DY", # key for root file with response matrix
                           bin_def, # 
                           variable,
                           regularisation_mode, 
                           bias,
                           density_mode)

In [ ]:
#detUnfold.checkMatrixCond()

In [ ]:
detUnfold.set_input_hist(use_MC_input=False)
detUnfold.set_bkgs()
#detUnfold.subFake() # Subract DY fake (out of fiducial volume at gen level), subUnfoldFake()

In [ ]:
if doSys == True:
    
    PDF_sys_list = ["PDFerror{:0>3}".format(str(i)) for i in range(1,101)]
    
    if channel == "mm" :
            
        systematics = {
            "Type_2": ["IdSFUp", "IdSFDown", 
                       "IsoSFUp","IsoSFDown", 
                       "trgSFUp", "trgSFDown", 
                       "PUUp", "PUDown", 
                       "bvetoUp", "bvetoDown", 
                       "ScaleABUp", "ScaleABDown", "ScaleAUp", "ScaleADown", "ScaleBUp", "ScaleBDown", 
                       "AlphaSUp", "AlphaSDown",
                       "LepMomScaleUp","LepMomScaleDown"
                      ],
        }
        systematics["Type_2"] += PDF_sys_list
    
    if channel == "ee" :
        
            systematics = {
                "Type_2": ["IDSF_up", "IDSF_down"],
                "Type_1": ["scale_up", "scale_down"]   # lepton momentum scale
            }
            
    for sys_type, sys_names in systematics.items():
        for sys_name in sys_names:
            
            detUnfold.set_systematics(sys_type, sys_name) # Currently, response matrix set here
            
            detUnfold.set_input_hist(False, sys_type, sys_name)
            detUnfold.set_bkgs(sys_type, sys_name) 
            #detUnfold.subFake("Detector_DY_Fake", sysType, sysName)

In [ ]:
do_partial_reg = False

# apply regularization for the pt distribution of the lass mass region
if variable == "[dipt-dimass]" :
    do_partial_reg = True

detUnfold.do_unfold(do_partial_reg) # nominal unfold option for partial regularization

In [ ]:
detUnfold.doAcceptance(isFSR=False)
detUnfold.closeOutFiles()
detUnfold.combineOutFiles()